<a href="https://colab.research.google.com/github/akua-m/ee109_final_project/blob/master/single_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#install all the required dependency libraries
!pip install tensorflow==1.4.0 #for prediction
!pip install numpy  #for matrix multiplication
!pip install pandas #define the data structures
!pip install matplotlib #for visualization
!pip install scikit-learn #for normalizing our data(scaling)

#importing the libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import io
%matplotlib inline

from tensorflow.python.client import timeline

In [3]:
#loading the dataset
from google.colab import files
uploaded = files.upload()

Saving btc.csv to btc (1).csv


In [4]:
#decoding the files as uploaded will be a dictionary of keys (the file names) and values (the encoded file objects)
btc = pd.read_csv(io.StringIO(uploaded['btc.csv'].decode('utf-8')))
btc.head()

,Date,Symbol,Open,High,Low,Close,Volume From,Volume To
0,5/26/2018,BTCUSD,7459.11,7640.46,7380.00,7520.00,2722.80,2.042265e+07
1,5/25/2018,BTCUSD,7584.15,7661.85,7326.94,7459.11,8491.93,6.342069e+07
2,5/24/2018,BTCUSD,7505.00,7734.99,7269.00,7584.15,11033.72,8.293137e+07
3,5/23/2018,BTCUSD,7987.70,8030.00,7433.19,7505.00,14905.99,1.148104e+08
4,5/22/2018,BTCUSD,8393.44,8400.00,7950.00,7987.70,6589.43,5.389753e+07


In [5]:
#selecting only the column that we are going to use in the prediction process
#then extracting one window of data from the column
data_to_use=btc['Close'].values
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_to_use.reshape(-1, 1))

window_data = scaled_data[1018:1026] # first element in array is most recent value
window_data

array([[-0.59011993],
       [-0.58965256],
       [-0.59124217],
       [-0.5905213 ],
       [-0.59232481],
       [-0.58745033],
       [-0.58735262],
       [-0.58673737]])

In [0]:
#import these arrays after training the network
#file_loc = "C:\Users\Akua McLeod\Documents\EE 109\weights_biases\"
file_loc = "/content/"


#load biases
bias_forget = np.loadtxt(fname = file_loc + "bias_forget_final")
bias_input = np.loadtxt(fname = file_loc + "bias_input_final")
bias_memory_cell = np.loadtxt(fname = file_loc + "bias_memory_cell_final")
bias_output = np.loadtxt(fname = file_loc + "bias_output_final")
bias_output_layer = np.loadtxt(fname = file_loc + "bias_output_layer_final")

# load weights
weights_forget_gate = np.loadtxt(fname = file_loc + "weights_forget_gate_final")
weights_forget_hidden = np.loadtxt(fname = file_loc + "weights_forget_hidden_final")
weights_input_gate = np.loadtxt(fname = file_loc + "weights_input_gate_final")
weights_input_hidden = np.loadtxt(fname = file_loc + "weights_input_hidden_final")
weights_memory_cell = np.loadtxt(fname = file_loc + "weights_memory_cell_final")
weights_memory_cell_hidden = np.loadtxt(fname = file_loc + "weights_memory_cell_hidden_final")
weights_output_gate = np.loadtxt(fname = file_loc + "weights_output_gate_final")
weights_output_hidden = np.loadtxt(fname = file_loc + "weights_output_hidden_final")
weights_output = np.loadtxt(fname = file_loc + "weights_output_final")

inputs = tf.placeholder(tf.float32)

batch_size = 1 #how many windows of data we are passing at once
window_size = 7 #how big window_size is (Or How many days do we consider to predict next point in the sequence)
hidden_layer = 256 #How many units do we use in LSTM cell


# convert to tensor objects and reshape
bias_forget = tf.reshape(tf.convert_to_tensor(bias_forget, dtype=tf.float32), [256])
bias_input = tf.reshape(tf.convert_to_tensor(bias_input, dtype=tf.float32), [256])
bias_memory_cell = tf.reshape(tf.convert_to_tensor(bias_memory_cell, dtype=tf.float32), [256])
bias_output = tf.reshape(tf.convert_to_tensor(bias_output, dtype=tf.float32), [256])
bias_output_layer = tf.reshape(tf.convert_to_tensor(bias_output_layer, dtype=tf.float32), [1])
weights_forget_gate = tf.reshape(tf.convert_to_tensor(weights_forget_gate, dtype=tf.float32), [1, 256])
weights_forget_hidden = tf.reshape(tf.convert_to_tensor(weights_forget_hidden, dtype=tf.float32), [256, 256])
weights_input_gate = tf.reshape(tf.convert_to_tensor(weights_input_gate, dtype=tf.float32), [1, 256])
weights_input_hidden = tf.reshape(tf.convert_to_tensor(weights_input_hidden, dtype=tf.float32), [256, 256])
weights_memory_cell_hidden = tf.reshape(tf.convert_to_tensor(weights_memory_cell_hidden, dtype=tf.float32), [256, 256])
weights_memory_cell = tf.reshape(tf.convert_to_tensor(weights_memory_cell, dtype=tf.float32), [1, 256])
weights_output_gate = tf.reshape(tf.convert_to_tensor(weights_output_gate, dtype=tf.float32), [1, 256])
weights_output_hidden = tf.reshape(tf.convert_to_tensor(weights_output_hidden, dtype=tf.float32), [256, 256])
weights_output = tf.reshape(tf.convert_to_tensor(weights_output, dtype=tf.float32), [256, 1])


#function to compute the gate states
def LSTM_cell(input, output, state):
    
    input_gate = tf.sigmoid(tf.matmul(input, weights_input_gate) + tf.matmul(output, weights_input_hidden) + bias_input)
    
    forget_gate = tf.sigmoid(tf.matmul(input, weights_forget_gate) + tf.matmul(output, weights_forget_hidden) + bias_forget)
    
    output_gate = tf.sigmoid(tf.matmul(input, weights_output_gate) + tf.matmul(output, weights_output_hidden) + bias_output)
    
    memory_cell = tf.tanh(tf.matmul(input, weights_memory_cell) + tf.matmul(output, weights_memory_cell_hidden) + bias_memory_cell)
    
    state = state * forget_gate + input_gate * memory_cell
    
    output = output_gate * tf.tanh(state)
    return state, output

#we now define one run for the network
batch_state = np.zeros([1, hidden_layer], dtype=np.float32) 
batch_output = np.zeros([1, hidden_layer], dtype=np.float32)

#for each point in the window we are feeding that into LSTM to get next output
for ii in range(window_size):
    batch_state, batch_output = LSTM_cell(tf.reshape(inputs[ii], [-1, 1]), batch_state, batch_output)
        
#last output is conisdered and used to get a prediction
prediction = (tf.matmul(batch_output, weights_output) + bias_output_layer)

In [16]:
#we now run one iteration of the network
#session = tf.Session()
#o = session.run([prediction], feed_dict={inputs:window_data})



with tf.Session() as session:
  options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
  run_metadata = tf.RunMetadata()

  o = session.run([prediction], feed_dict={inputs:window_data}, options=options, run_metadata=run_metadata)

  fetched_timeline = timeline.Timeline(run_metadata.step_stats)
  chrome_trace = fetched_timeline.generate_chrome_trace_format()
  with open('timeline_step.json', 'w') as f:
    f.write(chrome_trace)

o

[array([[-0.58934075]], dtype=float32)]